In [55]:
from qick import *
from qick.averager_program import *
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import csv
from datetime import date
from scipy.optimize import curve_fit
from scipy.optimize import leastsq,minimize
from scipy import stats
from sklearn import svm
import warnings
warnings.filterwarnings('ignore')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [56]:
soc = QickSoc(bitfile="q3diamond.bit")
soccfg = soc
print(soccfg)


QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 349.997, RF reference 245.760

	7 signal generator channels:
	0:	axis_sg_int4_v1 - tProc output 0, envelope memory 4096 samples
		DAC tile 2, ch 0, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	1:	axis_sg_int4_v1 - tProc output 1, envelope memory 4096 samples
		DAC tile 2, ch 1, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	2:	axis_sg_int4_v1 - tProc output 2, envelope memory 4096 samples
		DAC tile 2, ch 2, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	3:	axis_sg_int4_v1 - tProc output 3, envelope memory 4096 samples
		DAC tile 2, ch 3, 16-bit DDS, fabric=430.080 MHz, fs=1720.320 MHz
	4:	axis_signal_gen_v6 - tProc output 4, envelope memory 65536 samples
		DAC tile 3, ch 0, 32-bit DDS, fabric=599.040 MHz, fs=9584.640 MHz
	5:	axis_signal_gen_v6 - tProc output 5, envelope memory 65536 samples
		DAC tile 3, ch 1, 32-bit DDS, fabric=599.040 MHz, fs=9584.640 MHz
	6:	axis_sg_mux4_v2 - tProc output 6, envelope mem

In [57]:
for gen in soc.gens:
    print(gen)
    if gen.HAS_MIXER:
        print(gen.get_mixer_freq())
    if hasattr(gen, 'get_freq'):
        print([gen.get_freq(i) for i in range(4)])

500.0000000000094
500.0000000000094
500.0000000000094
500.0000000000094
500.0000000000094
[0.0, 0.0, 0.0, 0.0]


In [32]:
class ContinuousMixerProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg  
        
        res_ch = cfg["qubit_ch"]
        self.declare_gen(ch=res_ch, nqz=2, mixer_freq=cfg["mixer_freq"], ro_ch=0)
        
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=0)
        gain = cfg["pulse_gain"]
        
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=0, gain=gain)
        self.set_pulse_registers(ch=res_ch, style="const", length=cfg["length"], mode="periodic")
        
        self.synci(200)
    
    def body(self):
        
        self.pulse(ch=self.cfg["qubit_ch"])
        self.wait_all()

In [34]:
def MixerContinuousSignal(mix_freq, freq, gain):
    config={"qubit_ch":2,
            "reps":1,
            "relax_delay":1.0,
            "length":1000,
            "pulse_gain":gain,
            "pulse_freq":freq,
            "soft_avgs":1,
            "mixer_freq":mix_freq
           }

    prog =ContinuousMixerProgram(soccfg, config)
    avgq, avgi = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

In [35]:
MixerContinuousSignal(5000, 800, 1000)

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
soc.reset_gens()

# 

# 

# 

# 

# 

# 

# 

# 

# Find out how to send pulses with different lengths

In [60]:
class LoopbackProgram(AveragerProgram):
    def __init__(self,soccfg,cfg):
        super().__init__(soccfg,cfg)

    def initialize(self):
        cfg=self.cfg   
        self.declare_gen(ch=cfg["res_ch"], nqz=1, 
                         mixer_freq=cfg["mixer_freq"], 
                         mux_freqs=cfg["pulse_freqs"], 
                         mux_gains=cfg["pulse_gains"], 
                         ro_ch=cfg["ro_chs"][0])
        
        for iCh, ch in enumerate(cfg["ro_chs"]):
            self.declare_readout(ch=ch, freq=cfg["pulse_freqs"][iCh], length=cfg["readout_length"], gen_ch=cfg["res_ch"])

        self.synci(200)

    def body(self):
        cfg=self.cfg
        self.set_pulse_registers(ch=cfg["res_ch"], style="const", length=cfg["length"], mask=[1,2,3])
        self.trigger(adcs=self.ro_chs, pins=[0],adc_trig_offset=self.cfg["adc_trig_offset"])
        self.pulse(ch=self.cfg["res_ch"],t=0)

        self.waiti(0, self.cfg["adc_trig_offset"]+self.cfg["readout_length"]) 
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))
        self.sync_all(0)

config={"res_ch":6,
        "mixer_freq":1000.0,
        "ro_chs":[0,1,2,3],
        "reps":100,
        "relax_delay":10,
        "pulse_style": "const",
        "length":2**15-1,
        "readout_length":300,
        "pulse_freqs": [100, 300, 500, 700],
        "pulse_gains": [1.0, -0.8, 0.6, -0.4],
        "adc_trig_offset": 50,
        "soft_avgs":100
       }

prog =LoopbackProgram(soccfg, config)
avgi, avgq = prog.acquire(soc, load_pulses=True, progress=True, debug=False)

  0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
soc.reset_gens()